In [2]:
import os
import re
import argparse
import pickle

import pandas as pd
import torch

# 데이터 전처리

In [98]:
df = pd.read_csv('../dataset/Haircare/small_df.csv')

In [124]:
columns_list = df.columns.to_list()
for i in range(0, len(columns_list), 5):
    print(columns_list[i:i+5])

['collection', 'purchaseConditionInfos', 'rank', 'adId', 'id']
['parentId', 'attributeValueSeqs', 'attributeValue', 'characterValueSeqs', 'characterValue']
['productTitle', 'mallNo', 'mallId', 'isBrandStore', 'isNaverPay']
['isMblNaverPay', 'nPayPcType', 'nPayMblType', 'saleTp', 'mallInfo']
['dlvryCd', 'isAdult', 'hasVideo', 'deliveryFeeContent', 'hasCouponContent']
['isHotDeal', 'purchaseCnt', 'priceUnit', 'mobileLowPrice', 'imgColorCd']
['lowPrice', 'price', 'imgSgnt', 'checkOutReviewCount', 'category2Name']
['descriptionOrder', 'category2Id', 'category1Id', 'category3Id', 'category1Name']
['imgVersion', 'prchCondInfo', 'brand', 'lowPriceByMallNo', 'imageUrl']
['isLgtModelMat', 'smryReview', 'category3Name', 'hasLowestCardPrice', 'openDate']
['mobilePrice', 'additionalImageCount', 'reviewCountSum', 'scoreInfo', 'maker']
['mallCount', 'reviewCount', 'productName', 'prodTp', 'mpTp']
['overseaTp', 'imgSz', 'isMisImg', 'wdTp', 'dlvryLowPrice']
['buyPointContent', 'hasEventContent', 'hasB

In [10]:
duplicated_rows = df[df.duplicated(subset='상품명', keep=False)]
duplicated_rows.to_csv('duplicated_rows.csv', index=False, encoding='utf-8-sig')

# 그래프 생성 코드

In [2]:
from multisage.builder import PandasGraphBuilder

# 그래프의 기본적인 노드와 엣지만 생성
graph_builder = PandasGraphBuilder()
graph_builder.add_entities(genres, 'genre_id', 'genre')
graph_builder.add_entities(movies, 'movie_id', 'movie')
graph_builder.add_binary_relations(merged_ratings, 'genre_id', 'movie_id', 'define')
graph_builder.add_binary_relations(merged_ratings, 'movie_id', 'genre_id', 'define-by')
g = graph_builder.build()

ModuleNotFoundError: No module named 'multisage'

In [ ]:
g

In [ ]:
# 그래프 데이터 변형
g.nodes['genre'].data['id'] = torch.LongTensor(genres['genre_id'].cat.codes.values)
movies = pd.DataFrame(movies).astype({'year': 'category'})
genre_columns = movies.columns.drop(['movie_id', 'title', 'year', 'genre'])
movies[genre_columns] = movies[genre_columns].fillna(False).astype('bool')
g.nodes['movie'].data['year'] = torch.LongTensor(movies['year'].cat.codes.values)
g.nodes['movie'].data['genre'] = torch.FloatTensor(movies[genre_columns].values)
g.edges['define'].data['rating'] = torch.LongTensor(merged_ratings['rating'].values)
g.edges['define-by'].data['rating'] = torch.LongTensor(merged_ratings['rating'].values)

In [ ]:
# 그래프 저장

import dgl

output_path = 'graph_data.dgl'
dgl.save_graphs(output_path, [g])

# 테스트 용 Requset 코드 시작

In [28]:
from requests import request
from requests.compat import *
from bs4 import BeautifulSoup
from user_agent import generate_user_agent
import pandas as pd
import math
import time
import json
from pprint import pprint

In [69]:
url = 'https://search.shopping.naver.com/search/category/100000920?catId=50000197&frm=NVSHMDL&origQuery&pagingIndex=81&pagingSize=80&productSet=model&query&sort=rel&timestamp=&viewType=list'
resp = request('GET',url=url)
dom = BeautifulSoup(resp.text,'html.parser')

#print(dom.prettify())

script_tag = dom.find("script", {"id": "__NEXT_DATA__"})
json_data =json.loads(script_tag.text)
pprint(json_data)

{'assetPrefix': 'https://ssl.pstatic.net/shoppingsearch/static/pc/pc-230803-112911',
 'buildId': 'p2RdUPor_kjbn0ilNfl8D',
 'dynamicIds': [91117, 53502],
 'gssp': True,
 'isFallback': False,
 'page': '/search/category/[categoryId]',
 'props': {'__N_SSP': True,
           'pageProps': {'banners': [{'dataSeq': '1',
                                      'imageUrl': 'https://ssl.pstatic.net/shoppingsearch/static/icons/grocery/grocery_normal_banner_pc.png',
                                      'mallSeq': '99538',
                                      'modifyDate': '',
                                      'newWindow': True,
                                      'targetUrl': 'https://nid.naver.com/membership/join',
                                      'templateSeq': '1',
                                      'title': '멤버십 회원이라면, 장보기 최대 10% 적립'}],
                         'bucketId': '',
                         'cartProductCount': 0,
                         'dateStatus': {'date': '2023-08-

### 테스트 용 Requset 코드 끝

In [65]:
itemList = json_data['props']['pageProps']['initialState']['products']['list']
pprint(itemList)

[{'item': {'additionalImageCount': 0,
           'atmtTag': '',
           'attributeValue': '세트구성_M|세트구성_M|주요제품특징_M|주요제품특징_M|주요제품특징_M',
           'attributeValueSeqs': '10018513 10018513 10019300 10019300 10019300',
           'brand': '포레스트스토리',
           'brandNo': '232202',
           'buyPointContent': '^^^^^0^0',
           'cardContent': '',
           'category1Id': '50000002',
           'category1Name': '화장품/미용',
           'category2Id': '50000198',
           'category2Name': '헤어케어',
           'category3Id': '50000305',
           'category3Name': '헤어케어세트',
           'category4Id': '',
           'category4Name': '',
           'category4NameOrg': '',
           'categoryLevel': '3',
           'characterValue': '샴푸|트리트먼트|머릿결개선|향|촉촉함(수분공급)|1개',
           'characterValueSeqs': 'M10378821 M10378824 M10818751 M10818696 '
                                 'M10818653 P20042810',
           'checkOutReviewCount': 0,
           'chnlSeq': '',
           'collection': 'product'

In [62]:
items = [item_dict['item'] for item_dict in itemList[:5]]
item0, item1, item2, item3, item4 = items
item0

{'collection': 'product',
 'purchaseConditionInfos': [{'seq': '20042810',
   'condition': '1개',
   'count': '3',
   'lowPrice': '53780',
   'deliveryLowPrice': '56780',
   'unitPrice': '',
   'unit': '',
   'deliveryUnitPrice': '',
   'crUrl': 'https://cr.shopping.naver.com/adcr.nhn?x=Bn9MvC6d3OHgTrA0dMRKkP%2F%2F%2Fw%3D%3Dsw87PVQcUjLI8DmDKZDdtbvMPqZG%2FuZgvyNwFZJbzZ94NUoEqZCKHXsTTWg%2Bx6NB7BefMj2nAtMqkqGx73mMrGHhJb%2BDz5u%2F5EYl5VrYwsRN7YgImvtronwwdVj%2BGi2s4gg4IT3IjLyxPtul2iPz6HU9n8ka817ecM7U%2ByG4%2F5s0UhQwu46TIF9QHp%2FAh%2FIdX8ADBrrdtbZqH9xDsJ%2BemeDQjp6HY9fSNtKZ6NmqoK4nDpmNu2Rs3tEPrV0ZTODsb5ePKZ%2BA78%2BUNFD2FSnJuISnZUH89auG35rgVkHbxz4g1iAhzqsp0WIhhMgBgaIfk3W2elagPx3FhMxVxzH4TTMf6ueVlPfNqigfBZlDhAlTPgtjlWymUaED%2BAD3g75X5LSBnT7hA5H2QnEPljPchyIujqsmYSFAJXNm719nreGnFO4NKsK2SYlqsov7dfGT6noXlZUUWMrVRSgX%2BiABZZ3b1YTHweHyhLhZKMQdsJlfBokNhLzUhhNT0Z6yNXrbfY86KQZqYvDCcDPeXp8d10KDRu7haSN3ntR9e4YRsDUzBeM%2BV4IvZHKiplNXbCm851X4v%2FUDxzCzoMsWrD0w%2B8YC3D1PiRK8p6hyknJPdre4NO6GX498j%2FsBVfw67rKz

In [63]:
def compare_dicts(dict_list):
    # 첫 번째 딕셔너리를 기준으로 비교
    base_dict = dict_list[0]

    # 나머지 딕셔너리들과 첫 번째 딕셔너리를 비교
    for i, compare_dict in enumerate(dict_list[1:], start=1):
        print(f"Comparing item0 with item{i}:")

        for key, value in base_dict.items():
            if compare_dict[key] != value:
                print(f"Different value for key '{key}': item0={value}, item{i}={compare_dict[key]}")

# itemList에 있는 딕셔너리들을 비교
compare_dicts([item0, item1, item2, item3, item4])


Comparing item0 with item1:
Different value for key 'purchaseConditionInfos': item0=[{'seq': '20042810', 'condition': '1개', 'count': '3', 'lowPrice': '53780', 'deliveryLowPrice': '56780', 'unitPrice': '', 'unit': '', 'deliveryUnitPrice': '', 'crUrl': 'https://cr.shopping.naver.com/adcr.nhn?x=Bn9MvC6d3OHgTrA0dMRKkP%2F%2F%2Fw%3D%3Dsw87PVQcUjLI8DmDKZDdtbvMPqZG%2FuZgvyNwFZJbzZ94NUoEqZCKHXsTTWg%2Bx6NB7BefMj2nAtMqkqGx73mMrGHhJb%2BDz5u%2F5EYl5VrYwsRN7YgImvtronwwdVj%2BGi2s4gg4IT3IjLyxPtul2iPz6HU9n8ka817ecM7U%2ByG4%2F5s0UhQwu46TIF9QHp%2FAh%2FIdX8ADBrrdtbZqH9xDsJ%2BemeDQjp6HY9fSNtKZ6NmqoK4nDpmNu2Rs3tEPrV0ZTODsb5ePKZ%2BA78%2BUNFD2FSnJuISnZUH89auG35rgVkHbxz4g1iAhzqsp0WIhhMgBgaIfk3W2elagPx3FhMxVxzH4TTMf6ueVlPfNqigfBZlDhAlTPgtjlWymUaED%2BAD3g75X5LSBnT7hA5H2QnEPljPchyIujqsmYSFAJXNm719nreGnFO4NKsK2SYlqsov7dfGT6noXlZUUWMrVRSgX%2BiABZZ3b1YTHweHyhLhZKMQdsJlfBokNhLzUhhNT0Z6yNXrbfY86KQZqYvDCcDPeXp8d10KDRu7haSN3ntR9e4YRsDUzBeM%2BV4IvZHKiplNXbCm851X4v%2FUDxzCzoMsWrD0w%2B8YC3D1PiRK8p6hyknJPdre4NO6GX498j%2FsBV

# 데이터 수집 

In [73]:
import pandas as pd

def makePageDF(category_id, page_id, pagingsize):
    url = 'https://search.shopping.naver.com/search/category/' + str(category_id)
    params = {
        'pagingIndex' : str(page_id),
        'pagingSize' : str(pagingsize),
        'productSet' : 'model'
    }
    headers = {'User-Agent': 'Yeti',}
    resp = request('GET',url=url, params=params, headers=headers)
    print(resp.request.url)
    dom = BeautifulSoup(resp.text, 'html.parser')
    script_tag = dom.find("script", {"id": "__NEXT_DATA__"})
    data = json.loads(script_tag.text)
    itemList = data['props']['pageProps']['initialState']['products']
    
    
    # 각 상품의 정보를 저장할 리스트를 생성합니다.
    products_info = []

    for item in itemList['list'][:80]:
        products_info.append(item['item'])
    
    # 딕셔너리의 리스트를 데이터프레임으로 변환합니다.
    df = pd.DataFrame(products_info)

    return df

In [79]:
import pandas as pd

df = pd.DataFrame()

for k in range(100):
    new_df = makePageDF('100000921',k,80)
    df = pd.concat([df, new_df], ignore_index=True)

df


https://search.shopping.naver.com/search/category/100000921?pagingIndex=0&pagingSize=80&productSet=model
https://search.shopping.naver.com/search/category/100000921?pagingIndex=1&pagingSize=80&productSet=model
https://search.shopping.naver.com/search/category/100000921?pagingIndex=2&pagingSize=80&productSet=model
https://search.shopping.naver.com/search/category/100000921?pagingIndex=3&pagingSize=80&productSet=model
https://search.shopping.naver.com/search/category/100000921?pagingIndex=4&pagingSize=80&productSet=model
https://search.shopping.naver.com/search/category/100000921?pagingIndex=5&pagingSize=80&productSet=model
https://search.shopping.naver.com/search/category/100000921?pagingIndex=6&pagingSize=80&productSet=model
https://search.shopping.naver.com/search/category/100000921?pagingIndex=7&pagingSize=80&productSet=model
https://search.shopping.naver.com/search/category/100000921?pagingIndex=8&pagingSize=80&productSet=model
https://search.shopping.naver.com/search/category/10000

,collection,purchaseConditionInfos,rank,adId,id,parentId,attributeValueSeqs,attributeValue,characterValueSeqs,characterValue,...,isAdultExpsRstct,videoId,gdid,diffDeliveryFeeContent,eventContent,cardContent,shopNNo,dummy,crUrl,crUrlMore
0,product,None,1,nad-a001-02-000000190571337,32792384618,32792384618,10018513 10018513 10018513 10019300 10019300 1...,세트구성_M|세트구성_M|세트구성_M|주요제품특징_M|주요제품특징_M|주요제품특징_M,M10378821 M10378822 M10378824 M10818751 M10818...,샴푸|린스|트리트먼트|머릿결개선|향|촉촉함(수분공급)|1개,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,product,None,2,nad-a001-02-000000148770537,28269721554,28269721554,10011700 10011700 10011700 10011700 10011700 1...,타입_M|타입_M|타입_M|타입_M|타입_M|제품형태_M|세부제품특징_M|세부제품특...,M10378838 M10378839 M10378842 M10378843 M10378...,천연샴푸|비듬샴푸|두피샴푸|한방샴푸|퍼퓸샴푸|펌프형|저자극|촉촉함(수분공급)|각질케...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,product,None,3,nad-a001-02-000000148770535,28269744554,28269744554,10035978 10035988 10011700 10011701 10019299 1...,두피타입_M|모발타입_M|타입_M|제품형태_M|주요제품특징_M|주요제품특징_M|주요...,M11402182 M10378882 M10378830 M10276027 M10818...,모든두피용|모든 모발용|일반샴푸|펌프형|머릿결개선|세정력|풍부한 거품|상쾌함|파라벤...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,product,"[{'seq': '20131651', 'condition': '1개', 'count...",1,NaN,40292201619,40292201619,10035978 10035988 10011700 10011700 10011701 1...,두피타입_M|모발타입_M|타입_M|타입_M|제품형태_M|주요제품특징_M|주요제품특징...,M11402182 M10378882 M10378830 M10378837 M10276...,모든두피용|모든 모발용|일반샴푸|탈모샴푸|펌프형|저자극|세정력|탈모케어|트러블케어|...,...,0.0,,00000009_0009619a3493,,,,0,,https://cr.shopping.naver.com/adcr.nhn?x=HzF9P...,https://cr.shopping.naver.com/adcr.nhn?x=zu2w7...
4,product,"[{'seq': '20131651', 'condition': '1개', 'count...",2,NaN,5765532401,5765532401,10019371 10019371 10019371 10011545 10035958 1...,세부제품특징_M|세부제품특징_M|세부제품특징_M|용량_M|구성_M|용기형태_M,M10818653 M10527720 M10527718 M10276019 M10415...,촉촉함(수분공급)|윤기부여|영양공급|150ml(g)|본품|튜브형|1개|2개|3개|4...,...,0.0,,00000009_000157a70af1,,,,0,,https://cr.shopping.naver.com/adcr.nhn?x=8Ofmz...,https://cr.shopping.naver.com/adcr.nhn?x=lmuG0...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7995,product,"[{'seq': '20131651', 'condition': '1개', 'count...",7916,NaN,26026509522,26026509522,10011700 10011701 10019304 10019304 10019374 1...,타입_M|제품형태_M|주요제품특징_M|주요제품특징_M|세부제품특징_M|세부제품특징_...,M10378837 M10276027 M10718225 M10378945 M10818...,탈모샴푸|펌프형|세정력|탈모케어|촉촉함(수분공급)|두피케어|머릿결개선|본품|샴푸|5...,...,0.0,,00000009_00060f4d04d2,,,,0,,https://cr.shopping.naver.com/adcr.nhn?x=dOLct...,https://cr.shopping.naver.com/adcr.nhn?x=4xK5v...
7996,product,"[{'seq': '20131651', 'condition': '1개', 'count...",7917,NaN,32539409627,32539409627,10011700 10011701 10019299 10019299 10019369 1...,타입_M|제품형태_M|주요제품특징_M|주요제품특징_M|세부제품특징_M|세부제품특징_...,M10378830 M10276027 M10718225 M10818718 M10459...,일반샴푸|펌프형|세정력|풍부한 거품|저자극|촉촉함(수분공급)|모발영양|본품|1000...,...,0.0,,00000009_0007937ffcdb,,,,0,,https://cr.shopping.naver.com/adcr.nhn?x=M4K%2...,https://cr.shopping.naver.com/adcr.nhn?x=PAA%2...
7997,product,"[{'seq': '20042810', 'condition': '1개', 'count...",7918,NaN,5707860387,5707860387,10019370,세부제품특징_M,M10527718 P20042810,영양공급|1개,...,0.0,,00000009_0001543709a3,,,,0,,https://cr.shopping.naver.com/adcr.nhn?x=E%2BZ...,https://cr.shopping.naver.com/adcr.nhn?x=gVVA7...
7998,product,"[{'seq': '20131651', 'condition': '1개', 'count...",7919,NaN,28474792556,28474792556,10011702 10019301 10019371 10011545,헤어타입_M|주요제품특징_M|세부제품특징_M|용량_M,M10718258 M10818751 M10527718 M10276020 P20131651,손상 모발용|머릿결개선|영양공급|200ml(g)|1개,...,0.0,,00000009_0006a13ada6c,,,,0,,https://cr.shopping.naver.com/adcr.nhn?x=%2BGq...,https://cr.shopping.naver.com/adcr.nhn?x=LIP6n...


In [97]:
df.to_csv('../dataset/Haircare/small_df.csv', index=False, encoding='utf-8-sig')

### 데이터 수집 끝